In [1]:
import networkx as nx
import json
import sys
import os
from termcolor import colored
from tqdm import tqdm
from IPython.display import clear_output

In [2]:
CLASSES_PATH = os.path.dirname(os.path.abspath('../../Classes/'))
if not (CLASSES_PATH in sys.path):
    sys.path.append(CLASSES_PATH)
from Classes.Files_Handler_Class import Files_Handler
from Classes.Resd_Network_Infos_Class import Resd_Network_Infos
from Classes.SIR_Diffusion_Model_Class import SIR_Diffusion_Model
from Classes.Get_Past_Results_Class import Get_Past_Results
from Classes.Network_Infos_Writer_Class import Network_Infos_Writer
from Classes.Bcolors_Class import Bcolors as bcolors


In [3]:
seed_size_list = [# [1, 4, 8, 12, 16, 20, 24, 28],
                  # [1, 4, 8, 12, 16, 20, 24, 28]
                  # [1, 3, 6, 9, 12, 15, 17, 19],
                  # [1, 3, 6, 9, 12, 15, 17, 19]
                  # [1, 3, 6, 9, 12, 15, 17, 19]
                  # [1, 3, 6, 9, 12, 15, 17, 19]
                  # [1, 2, 4, 6, 8, 10, 12, 14],
                  # [1, 2, 3, 4, 5, 6, 8, 10],
                  # [1, 2, 3, 4, 5],
                  # [1, 2, 3, 4, 5]
                  ]
datasets_list = [# '01 - Sanremo2016_final (55897 Node and 3 Layer)/Sanremo2016_final.edgeslist',
                 # '02 - C_H_N (45583 Node and 3 Layer)/C_H_N.edgeslist'
                 # '03 - higgs-reply_network (38918 Node and 20 Layer)/higgs-reply_network.edgeslist'
                 # '04 - 2 AS (33752 Node and 7 Layer)/2 AS.edgeslist'
                 # '05 - friendfeed_ita (21006 Node and 3 Layer)/friendfeed_ita.edgeslist',
                 # '06 - Figs S1-S5 (10000 Node and 5 Layer)/Figs S1-S5.edgeslist'
                 # '07 - fftwyt (6407 Node and 3 Layer)/fftwyt.edgeslist',
                 # '08 - Irvine (1663 Node and 51 Layer)/Irvine.edgeslist',
                 # '09 - Wainwright (217 Node and 36 Layer)/Wainwright.edges',
                 # '10 - realitycommons (84 Node and 5 Layer)/Relationshipsfromsurveys.edgelist'
                
                ]

root_file_path = 'D:/Masters thesis/Methods For Compare/Datasets/' 

color_list = ["light_red", "light_green", "light_yellow",
                    "light_blue","light_magenta", "light_cyan",
                    "blue", "red", "white", "green", "yellow",
                        "magenta", "cyan", ]
tqdm_color_list = ['blue', 'red', 'green', 'cyan', 'magenta', 'yellow', 'black', 'white']


files_handler_obj = Files_Handler()
SIR_diffusion_model_obj = SIR_Diffusion_Model()
resd_network_infos_obj = Resd_Network_Infos()


source_code_path = str(os.getcwd())
source_code_path = source_code_path.replace("\\", "/")


In [4]:
def draw_graphs_colored_by_attribute(graphs_of_network:list[nx.Graph], attribute:str):
    import matplotlib.pyplot as plt
    from matplotlib.pyplot import figure
    # create number for each group to allow use of colormap
    from itertools import count
    # get unique groups
    for j, graph in enumerate(graphs_of_network):
        if len(graph) > 0:
            groups = set(nx.get_node_attributes(graph, attribute).values())
            mapping = dict(zip(sorted(groups),count()))
            nodes = graph.nodes()
            colors = [mapping[graph.nodes[n][attribute]] for n in nodes]

            # drawing nodes and edges separately so we can capture collection for colobar
            pos = nx.spring_layout(graph)
            ec = nx.draw_networkx_edges(graph, pos, alpha=0.3)
            labels = nx.draw_networkx_labels(graph, pos=pos)
            nc = nx.draw_networkx_nodes(graph, pos, label=labels, nodelist=nodes, node_color=colors, cmap=plt.cm.jet)
            plt.colorbar(nc)
            plt.axis('off')
            plt.show()

def dsc(layer_l, ul, vl):
    nbrs_ul = list(nx.neighbors(layer_l, ul))
    nbrs_vl = list(nx.neighbors(layer_l, vl))
    if len(nbrs_ul) <= len(nbrs_vl):
        subscribe_uv = set([x for x in nbrs_ul if x in nbrs_vl])
    else:
        subscribe_uv = set([x for x in nbrs_vl if x in nbrs_ul])
    if len(subscribe_uv) <= 0:
        dsc_value = 0
    else:
        dsc_value = (2 * len(subscribe_uv)) / len(nbrs_ul + nbrs_vl)
    return dsc_value

def get_max_dice_node(layer_id: int, dice_mat:dict, node:str | int):
    max_similar_dice_node = None
    max_similar_dice_value = -1
    
    if node in dice_mat.keys() and len(dice_mat[node].keys()) > 0:
        max_similar_dice_node = str(max(dice_mat[node], key=dice_mat[node].get))
        max_similar_dice_value = dice_mat[node][max_similar_dice_node]
    for kk, dice_dict in dice_mat.items():
        if kk != node:
            if node in dice_dict.keys():
                if dice_dict[node] >= max_similar_dice_value:
                    max_similar_dice_node = kk
                    max_similar_dice_value = dice_dict[node]
        else:
            break

    return max_similar_dice_node

def get_group_in_and_out_degree(graph:nx.Graph, nodes_group:list):
    out_neighbors = []
    all_neighbors = []
    for item in nodes_group:
        neighbors = list(graph.neighbors(item))
        all_neighbors.extend(neighbors)
        neighbors = list(set(neighbors) - set(nodes_group))
        out_neighbors.extend(neighbors)

    in_neighbors = [x for x in all_neighbors if x not in out_neighbors]
    
    return len(in_neighbors)/2, len(out_neighbors)

def get_community_conductance(graph, community_nodes):
    e_in, e_out = get_group_in_and_out_degree(graph, community_nodes)
    community_conductance = e_out / ((2 * e_in) + e_out)
    return community_conductance

def community_similarity(layer_id, dice_3D_mat, community_a, community_b):
    similarity = -1
    dsc = 0
    for j, node_a in enumerate(community_a):
        for i, node_b in enumerate(community_b):
            if node_a in dice_3D_mat[layer_id].keys():
                if node_b in dice_3D_mat[layer_id][node_a].keys():
                    dsc += dice_3D_mat[layer_id][node_a][node_b]
    
    similarity = dsc / len(community_a)
    return similarity

def ews(graph:nx.Graph, node, community_nodes):
    ews_value = 0
    alpha = 0.1
    h = 2
    neighbors = nx.single_source_shortest_path_length(graph, node, cutoff=h)
    for kk, v in neighbors.items():
        if kk in community_nodes:
            a_hl_ij = len(list(nx.all_simple_paths(graph, source=node, target=kk, cutoff=h)))
            ews_value += ((alpha** v) * graph.degree[node]) * (a_hl_ij)

    return ews_value

def get_quota(k, n, v):
    quota = k * (n/v)
    return quota



In [5]:
for i, path in enumerate(datasets_list):
    file_path = root_file_path + path
    for item in seed_size_list[i]:
        print(path, item)
        seed_set_size = item
#--------------------------------------------------------------------------------------------------------------------------
        if file_path is None or file_path == '':
            sys.exit("File Selection Canceled !")
        file_info = files_handler_obj.get_file_path_info(file_path)
        network_name = file_info['name']
        network_type = file_info['type']
        network_path = file_info['path']
        if network_name == "":
            sys.exit("Dont Network Selection!")
#--------------------------------------------------------------------------------------------------------------------------
        get_past_results_obj = Get_Past_Results(file_info['path'], file_info['name'])
        network_infos_writer_obj = Network_Infos_Writer(file_info['path'], file_info['name'])
#--------------------------------------------------------------------------------------------------------------------------
        (
            network_layers_info,
            network_layers_nodes,
            entra_layer_edges,
            entra_layer_edges_features,
            inter_layer_edge,
        ) = resd_network_infos_obj.read_nodeFrom_layerFrom_nodeTo_layerTo(
            network_path, network_name, network_type
        )
#--------------------------------------------------------------------------------------------------------------------------
        print(file_info['name'], '\n')
        sum_of_layers_nodes = 0
        network_layers_count = len(network_layers_info)
        graphs_of_network = [None] * network_layers_count
        network_entier_edges = ""
        layers_nodes_infect_scale = []
        network_entier_nodes_list = []

        i = 0
        j = 0
        while i < network_layers_count:
            graphs_of_network[i] = nx.Graph()
            network_layers_nodes[i] = list(set(network_layers_nodes[i]))
            layers_nodes_infect_scale.append({})
            if len(network_layers_nodes[i]) > 0:
                # graphs_of_network[i].add_nodes_from(network_layers_nodes[i])
                graphs_of_network[i].add_edges_from(entra_layer_edges[i])

                nx.set_node_attributes(graphs_of_network[i], None, "community")

                nx.set_edge_attributes(graphs_of_network[i], 1, "weight")
                
                graphs_of_network[i].graph["id"] = i
                graphs_of_network[i].graph["node_num"] = graphs_of_network[i].number_of_nodes()
                graphs_of_network[i].graph["edge_num"] = graphs_of_network[i].number_of_edges()

                print(colored("Layer "  + str(i) + ": " + str(graphs_of_network[i].number_of_nodes()) + " Node And " +
                            str(graphs_of_network[i].number_of_edges()) + " Edge", color_list[j]))
                # print(colored(graphs_of_network[i].graph['k_shell_info'], color_list[i]))
                network_entier_nodes_list.extend(list(graphs_of_network[i].nodes))
                sum_of_layers_nodes += len(list(graphs_of_network[i].nodes))
            i += 1
            j += 1
            if j >= len(color_list):
                j = 0
                
        network_entier_nodes_list = list(set(network_entier_nodes_list))
        network_entier_nodes_count = len(network_entier_nodes_list)
        print()
        print("network entier nodes : " + colored(str(network_entier_nodes_count), "yellow"))
        CBIM_path = files_handler_obj.make_dir(file_info['path'], 'CBIM')
#--------------------------------------------------------------------------------------------------------------------------
        sorted_nodes, sorted_nodes_load_flag = get_past_results_obj.load_results_from_file(CBIM_path, 'sorted_nodes')
        print('sorted_nodes_load_flag: ', sorted_nodes_load_flag)
        if sorted_nodes_load_flag == False:
            sorted_nodes = []
            for i, graph in enumerate(graphs_of_network):
                sorted_nodes.append(sorted(graph.degree, key=lambda x: x[1], reverse=True))
#--------------------------------------------------------------------------------------------------------------------------
        if sorted_nodes_load_flag == False:
            network_infos_writer_obj. write_results_in_file(CBIM_path, 'sorted_nodes', sorted_nodes)

#--------------------------------------------------------------------------------------------------------------------------
        print(CBIM_path)
        dice_3D_mat, dice_3D_mat_load_flag = get_past_results_obj.load_results_from_file(CBIM_path, 'dice_3D_mat')
        print('dice_3D_mat_load_flag: ', dice_3D_mat_load_flag)
        if dice_3D_mat_load_flag == False:
            dice_3D_mat = []
            tqdm_i = 0
            for j, graph in enumerate(graphs_of_network):
                dice_3D_mat.append({})
                if graph.number_of_nodes() > 0:
                    if tqdm_i > len(tqdm_color_list) - 1:
                        tqdm_i = 0
                    with tqdm(sorted_nodes[j], unit=" Node", colour=tqdm_color_list[tqdm_i]) as t_nodes:
                        for i, node_u in enumerate(t_nodes):
                            if i < (len(sorted_nodes[j]) - 1):
                                c = i + 1
                                dice_3D_mat[j][node_u[0]] = {}
                                while c < len(sorted_nodes[j]):
                                    node_v = sorted_nodes[j][c]
                                    dice_value = dsc(graph, node_u[0], node_v[0])
                                    if dice_value > 0:
                                        dice_3D_mat[j][node_u[0]][node_v[0]] = dice_value
                                    c += 1
                    tqdm_i += 1

#--------------------------------------------------------------------------------------------------------------------------
        if dice_3D_mat_load_flag == False:
            network_infos_writer_obj.write_results_in_file(CBIM_path, 'dice_3D_mat', dice_3D_mat)

        communities_counter, communities_counter_load_flag = get_past_results_obj.load_results_from_file(CBIM_path, 'communities_counter')
        communities_groups, communities_groups_load_flag = get_past_results_obj.load_results_from_file(CBIM_path, 'communities_groups')
        print('communities_counter_load_flag: ', communities_counter_load_flag, '\ncommunities_groups_load_flag: ', communities_groups_load_flag)
#--------------------------------------------------------------------------------------------------------------------------
        if communities_counter_load_flag == False or communities_groups_load_flag == False:
            communities_counter = []
            communities_groups = []
            for j, graph in enumerate(graphs_of_network):
                communities_counter.append([])
                communities_groups.append([])
                if len(graph) > 0:
                    for i, node in enumerate(sorted_nodes[j]):
                        if i != 0:
                            max_similar_dice_node = get_max_dice_node(j, dice_3D_mat[j], node[0])
                            if max_similar_dice_node != None:
                                if graph.nodes[max_similar_dice_node]['community'] != None:
                                    graph.nodes[node[0]]['community'] = graph.nodes[max_similar_dice_node]['community']
                                    if not (node[0] in communities_groups[j][int(graph.nodes[max_similar_dice_node]['community'])]):
                                        communities_groups[j][int(graph.nodes[max_similar_dice_node]['community'])].append(node[0])
                                        communities_counter[j][int(graph.nodes[max_similar_dice_node]['community'])][1] += 1
                                else:
                                    communities_counter[j].append([(len(communities_counter[j])), 0])
                                    communities_groups[j].append([])
                                    graph.nodes[node[0]]['community'] = communities_counter[j][-1][0]
                                    graph.nodes[max_similar_dice_node]['community'] = communities_counter[j][-1][0]
                                    if not(node[0] in communities_groups[j][-1]):
                                        communities_groups[j][-1].append(node[0])
                                        communities_counter[j][-1][1] += 1
                                    if not(max_similar_dice_node in communities_groups[j][-1]):
                                        communities_groups[j][-1].append(max_similar_dice_node)
                                        communities_counter[j][-1][1] += 1
                        else:
                            communities_counter[j].append([(len(communities_counter[j])), 0])
                            communities_groups[j].append([])
                            graph.nodes[node[0]]['community'] = communities_counter[j][-1][0]
                            if not(node[0] in communities_groups[j][-1]):
                                communities_groups[j][-1].append(node[0])
                                communities_counter[j][-1][1] += 1
                            max_similar_dice_node = get_max_dice_node(j, dice_3D_mat[j], node[0])
                            if max_similar_dice_node != None:
                                graph.nodes[max_similar_dice_node]['community'] = graph.nodes[node[0]]['community']
                                if not(max_similar_dice_node in communities_groups[j][-1]):
                                    communities_groups[j][-1].append(max_similar_dice_node)
                                    communities_counter[j][-1][1] += 1

#--------------------------------------------------------------------------------------------------------------------------
        if communities_counter_load_flag == False or communities_groups_load_flag == False:
            for j, graph in enumerate(graphs_of_network):
                if len(graph) > 0:
                    for i, node in enumerate(sorted_nodes[j]):
                        if graph.nodes[node[0]]['community'] == None:
                            neighbors = list(nx.neighbors(graph, node[0]))
                            neighbor_community = None
                            neighbor_node = None
                            for item in neighbors:
                                if graph.nodes[item]['community'] != None:
                                    if neighbor_community != None:

                                        if graph.degree(item) > graph.degree(neighbor_node):
                                            neighbor_community = graph.nodes[item]['community']
                                            neighbor_node = item    
                                    else:
                                        neighbor_community = graph.nodes[item]['community']
                                        neighbor_node = item
                            
                            if neighbor_community is None:
                                communities_counter[j].append([(len(communities_counter[j])), 0])
                                communities_groups[j].append([])
                                graph.nodes[node[0]]['community'] = communities_counter[j][-1][0]
                                if not(node[0] in communities_groups[j][-1]):
                                    communities_groups[j][-1].append(node[0])
                                    communities_counter[j][-1][1] += 1
                            else:
                                graph.nodes[node[0]]['community'] = neighbor_community
                                if not(node[0] in communities_groups[j][int(neighbor_community)]):
                                    communities_groups[j][int(neighbor_community)].append(node[0])
                                    communities_counter[j][int(neighbor_community)][1] += 1
                    
#--------------------------------------------------------------------------------------------------------------------------
        draw_status = False
        if draw_status:
            draw_graphs_colored_by_attribute(graphs_of_network, 'community')
#--------------------------------------------------------------------------------------------------------------------------
        delta = 0.025
        if communities_counter_load_flag == False or communities_groups_load_flag == False:
            for j, graph in enumerate(graphs_of_network):
                if len(graph) > 0:
                    print(bcolors.HEADER + f'Layer {graph.graph["id"]}: {graph} communities: {len(communities_groups[j])}' + bcolors.ENDC)
                    print()
                    merging_flag = True
                    merging_flag = False
                    while(merging_flag == True):
                        for i, community_nodes in enumerate(communities_groups[j]):
                            if len(community_nodes) > 0:
                                community_conductance = get_community_conductance(graph, community_nodes)
                                community_scale = len(community_nodes)/graph.number_of_nodes()
                                community_merging_index = community_conductance * community_scale
                                if delta < community_merging_index:
                                    merging_flag = True
                                    communities_similarity = []
                                    for kk, community_nodes_b in enumerate(communities_groups[j]):
                                        if len(community_nodes_b) <= 0:
                                            communities_similarity.append(-1)
                                        else:
                                            if i != kk:
                                                communities_similarity.append(community_similarity(j, dice_3D_mat, community_nodes, community_nodes_b))
                                            else:
                                                communities_similarity.append(-1)
                                    max_similarity_community_index = communities_similarity.index(max(communities_similarity))
                                    for item in community_nodes:
                                        graph.nodes[item]['community'] = communities_counter[j][max_similarity_community_index][0]
                                    communities_groups[j][max_similarity_community_index].extend(communities_groups[j][i])
                                    communities_groups[j][i] = []
                                    communities_counter[j][max_similarity_community_index][1] = 0
                                    break

#--------------------------------------------------------------------------------------------------------------------------
        if communities_counter_load_flag == False or communities_groups_load_flag == False:
            network_infos_writer_obj.write_results_in_file(CBIM_path, 'communities_counter', communities_counter)
            network_infos_writer_obj.write_results_in_file(CBIM_path, 'communities_groups', communities_groups)
        draw_status = False
        if draw_status:
            draw_graphs_colored_by_attribute(graphs_of_network, 'community')
            
#--------------------------------------------------------------------------------------------------------------------------
        sorted_communities_ews, sorted_communities_ews_load_flag = get_past_results_obj.load_results_from_file(CBIM_path, 'sorted_communities_ews')
        print('sorted_communities_ews_load_flag: ', sorted_communities_ews_load_flag)
        if sorted_communities_ews_load_flag == False:
            communities_ews = []
            tqdm_i = 0
            for j, graph in enumerate(graphs_of_network):
                communities_ews.append({})
                if len(graph) > 0:
                    print(bcolors.HEADER + f'Layer {graph.graph["id"]}: {graph}' + bcolors.ENDC)
                    pbar = tqdm(total=len(communities_groups[j]))
                    if tqdm_i > len(tqdm_color_list) - 1:
                        tqdm_i = 0
                    pbar.colour = tqdm_color_list[tqdm_i]
                    tqdm_i += 1
                    pbar.unit = ' community'
                    p_itr = 1
                    for i, community_nodes in enumerate(communities_groups[j]):
                        if len(community_nodes) > 0:
                            communities_ews[j][i] = {}
                            for node in community_nodes:
                                communities_ews[j][i][node] = ews(graph, node, community_nodes)
                        pbar.set_description(f'community {p_itr}')
                        p_itr += 1
                        pbar.update(1)
                    pbar.close()

#--------------------------------------------------------------------------------------------------------------------------
        if sorted_communities_ews_load_flag == False:
            sorted_communities_ews = []
            pbar = tqdm(total=len(communities_ews))
            pbar.colour = 'green'
            pbar.unit = ' communities_ews'
            p_itr = 1
            for j, item in enumerate(communities_ews):
                sorted_communities_ews.append({})
                if len(item.keys()) > 0:
                    for kk, vv in item.items():
                        sorted_communities_ews[j][kk] = {kk: v for kk, v in sorted(vv.items(), key=lambda item: item[1], reverse=True)}
                pbar.set_description(f'communities_ews {p_itr}')
                p_itr += 1
                pbar.update(1)
            pbar.close()

#--------------------------------------------------------------------------------------------------------------------------
        if sorted_communities_ews_load_flag == False:
            network_infos_writer_obj.write_results_in_file(CBIM_path, 'sorted_communities_ews', sorted_communities_ews)

#--------------------------------------------------------------------------------------------------------------------------
        sorted_quota, sorted_quota_load_flag = get_past_results_obj.load_results_from_file(CBIM_path, 'sorted_quota')
        print('sorted_quota_load_flag: ', sorted_quota_load_flag)
#--------------------------------------------------------------------------------------------------------------------------

        if sorted_quota_load_flag == False:
            quota = {}
            community_counter = 0
            pbar = tqdm(total=len(sorted_communities_ews))
            pbar.colour = 'blue'
            pbar.unit = ' sorted_communities_ews'
            p_itr = 1
            for j, item in enumerate(sorted_communities_ews):
                quota[j]={}
                if len(item.keys()) > 0:
                    for kk, vv in item.items():
                        quota[j][kk] = get_quota(seed_set_size, len(vv.keys()), sum_of_layers_nodes)
                        community_counter += 1
                pbar.set_description(f'sorted_communities_ews {p_itr}')
                p_itr += 1
                pbar.update(1)
            pbar.close()
        # quota
#--------------------------------------------------------------------------------------------------------------------------
        if sorted_quota_load_flag == False:
            sorted_quota = []
            pbar = tqdm(total=community_counter)
            pbar.colour = 'red'
            pbar.unit = ' community'
            p_itr = 1
            for j in range(community_counter):
                max_quota_value = -1
                max_quota_cummunity = None
                max_quota_layer = None
                for i, _ in enumerate(quota.items()):
                    if len(quota[i].keys()) > 0:
                        for kk, vv in quota[i].items():
                            if  vv > max_quota_value :
                                max_quota_value = vv
                                max_quota_cummunity = kk
                                max_quota_layer = i
                del quota[max_quota_layer][max_quota_cummunity]
                sorted_quota.append((max_quota_layer, max_quota_cummunity, max_quota_value))
                pbar.set_description(f'community {p_itr}')
                p_itr += 1
                pbar.update(1)
            pbar.close()
                            
        # sorted_quota
#--------------------------------------------------------------------------------------------------------------------------
        if sorted_quota_load_flag == False:
            network_infos_writer_obj.write_results_in_file(CBIM_path, 'sorted_quota', sorted_quota)
#--------------------------------------------------------------------------------------------------------------------------
        seed_set = []
        seed_set_counter = 0

        for item in sorted_quota:
            if len(seed_set) >= seed_set_size:
                    break

            if type(list(sorted_communities_ews[item[0]].keys())[0]) == str:
                community_id = str(item[1])
            else:
                community_id = item[1]
            community_seed_node_quota = round(item[2])
            itr = iter(sorted_communities_ews[item[0]][community_id])
            community_seed_node_count = len(sorted_communities_ews[item[0]][community_id].keys())
            i = 0
            j = 0
            while i < community_seed_node_quota:
                seed_node = next(itr)
                if not(seed_node in seed_set):
                    seed_set.append(seed_node)
                    seed_set_counter += 1
                    i += 1
                j += 1
                if j >= community_seed_node_count:
                    break
                if len(seed_set) >= seed_set_size:
                    break
            if len(seed_set) >= seed_set_size:
                    break

        if len(seed_set) < seed_set_size:
            for item in sorted_quota:
                if type(list(sorted_communities_ews[item[0]].keys())[0]) == str:
                    community_id = str(item[1])
                else:
                    community_id = item[1]
                itr = iter(sorted_communities_ews[item[0]][community_id])
                community_seed_node_count = len(sorted_communities_ews[item[0]][community_id].keys())
                j = 0
                if round(item[2]) < 1:
                    while True:
                        seed_node = next(itr)
                        if not(seed_node in seed_set):
                            seed_set.append(seed_node)
                            seed_set_counter += 1
                            break
                        j += 1
                        if j > community_seed_node_count or len(seed_set) >= seed_set_size:
                            break
                if len(seed_set) >= seed_set_size:
                        break

        if len(seed_set) < seed_set_size:
            for item in sorted_quota:
                if type(list(sorted_communities_ews[item[0]].keys())[0]) == str:
                    community_id = str(item[1])
                else:
                    community_id = item[1]
                itr = iter(sorted_communities_ews[item[0]][community_id])
                community_seed_node_count = len(sorted_communities_ews[item[0]][community_id].keys())
                j = 0
                while True:
                    seed_node = next(itr)
                    if not(seed_node in seed_set):
                        seed_set.append(seed_node)
                        seed_set_counter += 1
                        break
                    j += 1
                    if j > community_seed_node_count or len(seed_set) >= seed_set_size:
                            break
                if len(seed_set) >= seed_set_size:
                        break


        print('Seedset size: ', seed_set_size)
        print('Seedset: ', seed_set)
#--------------------------------------------------------------------------------------------------------------------------
        print(f"Network name: {network_name}")
        print(f"Seed set size: {seed_set_size}")
        beta = 0.01
        landa = 0.7
        epoch = 10000

        infection = SIR_diffusion_model_obj.synchronous_SIR_multilayer_with_seed_set_model(graphs_of_network, seed_set, beta, landa, epoch, network_entier_nodes_list)

        sir_results_infos = {}
        sir_results_infos['infection'] = infection
        sir_results_infos['percentage'] = infection / network_entier_nodes_count
        sir_results_infos['seed_set'] = seed_set

        network_infos_writer_obj.write_results_in_file(CBIM_path, f'infection k={seed_set_size} beta={beta} landa{landa} epoch{epoch}', sir_results_infos)
        print(f"Network entier nodes count: {network_entier_nodes_count}")
        print(f"Infected nodes count: {infection}")
        print(f"Percentage of infection: {sir_results_infos['percentage']}")

#--------------------------------------------------------------------------------------------------------------------------
        clear_output(wait=False)
#--------------------------------------------------------------------------------------------------------------------------
print('Finished!')

Finished!
